This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
# get api key from your .env fileimport os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import json
import collections
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [38]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Assign URL to variable: url

start_date = '2017-01-01'
end_date = '2017-12-31'
collapse = 'daily'
# exclude_column_names = 'true'
column_index = 1

# second url approach: 

url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json' 
params = {    
    'api_key': API_KEY,     
    'start_date': start_date,     
    'end_date': end_date,
    'collapse': collapse,
    # 'column_index': '1',
    # 'column_index': '2'
}

# Package the request, send the request and catch the response: r
# r = requests.get(url)
r = requests.get(url, params=params)

# Store the text of the response

# print(r.text)
stocks_data2 = r.text

# print(json.dumps(r.text, indent=2))

# ============================================
    
# API Guide:
# https://blog.data.nasdaq.com/getting-started-with-the-nasdaq-data-link-api
# https://docs.data.nasdaq.com/
# https://help.data.nasdaq.com/article/497-the-api-is-not-working-for-me-what-should-i-do

# https://docs.data.nasdaq.com/docs/quick-start-examples-dd

ok


In [ ]:
with open('json_stocks2.json', 'w') as json_file:
    json_file.write(stocks_data2 + '\n')

with open('json_stocks2.json', 'r') as json_file:
    existing_data_str = json_file.read()
    
# ============IMPORTANT================

formatted_data = json.dumps(json.loads(existing_data_str), indent=2)
print(formatted_data)


In [14]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
# ============
# 2. Convert the returned JSON object into a Python dictionary.
# ============ 

import json

# Convert JSON string to a dictionary
data_dict = json.loads(formatted_data)

# Print the resulting dictionary
print(data_dict)

In [26]:
# ========
# 3. Calculate what the highest and lowest opening prices were for the stock in this period
# ========

column_names = data_dict['dataset_data']['column_names']
data = data_dict['dataset_data']['data']

# Find the highest and lowest prices
highest_price = {'Date': None, 'High': float('-inf')}
lowest_price = {'Date': None, 'Low': float('inf')}

for entry in data:
    date, high, low = entry[0], entry[2], entry[3]

    # Update highest price
    if high > highest_price['High']:
        highest_price['Date'] = date
        highest_price['High'] = high

    # Update lowest price
    if low < lowest_price['Low']:
        lowest_price['Date'] = date
        lowest_price['Low'] = low

# Display the results
print(f'Highest Price: {highest_price}\n')
print(f'Lowest Price: {lowest_price}\n')

Highest Price: {'Date': '2017-12-14', 'High': 53.54}

Lowest Price: {'Date': '2017-01-23', 'Low': 33.62}



In [44]:
# ============
# 4. What was the largest change in any one day (based on High and Low price)?
# ============

largest_change = {'Date': None, 'Change': 0}

for entry in data:
    date, high, low = entry[0], entry[2], entry[3]

    current_change = abs(high - low)

    # Update largest change
    if current_change > largest_change['Change']:
        largest_change['Date'] = date
        largest_change['Change'] = current_change

# Display the results
# 5. What was the largest change between any two days (based on Closing Price)
print(f'Largest Change: {largest_change}\n')

Largest Change: {'Date': '2017-05-11', 'Change': 2.8100000000000023}



In [37]:
# ============
# 5. What was the largest change between any two days (based on Closing Price)?
# ============

two_day_change = {'Date': None, 'Change': 0}

# Sort data by date in ascending order
sorted_data = sorted(data, key=lambda x: x[0])

for i in range(len(sorted_data) - 1):  # Iterate through pairs of consecutive entries
    date1, close1 = sorted_data[i][0], sorted_data[i][4]
    date2, close2 = sorted_data[i + 1][0], sorted_data[i + 1][4]

    current_change = abs(close2 - close1)

    # Update two_day_change
    if current_change > two_day_change['Change']:
        two_day_change['Date'] = f"{date1} to {date2}"
        two_day_change['Change'] = current_change

# Display the results
print('\nLargest Two-Day Change: ', two_day_change,'\n')


Largest Two-Day Change:  {'Date': '2017-08-08 to 2017-08-09', 'Change': 2.559999999999995} 



In [48]:
# ========
# 6. What was the average daily trading volume during this year?
# ========

total = 0
entry_count = 0

for entry in data:
    traded_volume = entry[6]
    total += traded_volume
    entry_count += 1

# print(total)
# print(entry_count)

print('\nAverage Trading Volume: ', total/entry_count,'\n')


Average Trading Volume:  89124.33725490196 

